In [35]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Data

In [37]:
import pandas as pd
data_path = '/content/drive/My Drive/AmbiCoref/AmbiCoref/Data/'

## Verbs


In [50]:
# Type ECO
ECO_all = pd.read_csv(data_path+"Verb_phrases/ECO.csv")
amuse_verbs = ECO_all['bored'].dropna()
see_verbs = ECO_all['saw'].dropna()

In [51]:
# Type ECS
ECS_all = pd.read_csv(data_path+"Verb_phrases/ECS.csv")
admire_verbs = ECS_all['liked'].dropna()
meet_verbs = ECS_all['met-with'].dropna()

In [52]:
#Type IC
IC_all = pd.read_csv(data_path+"Verb_phrases/IC.csv")
call_verbs = IC_all['called'].dropna()
call_passive = IC_all['called_passive'].dropna()
reasons_amb = IC_all['reason_ambig'].dropna()
reasons_unamb = IC_all['reason_unambig'].dropna()

In [53]:
# Type TOP
TOP_all = pd.read_csv(data_path+"Verb_phrases/TOP.csv")

def male_to_female(text):
  text1 = text.replace(" he "," she ")
  text2 = text1.replace(" him", " her")
  text3 = text2.replace(" his", " her")
  return text3

### Retrieve the NP pairs

In [54]:
names_df = pd.read_csv(data_path + 'NPs/Names.csv')
NPs_df = pd.read_csv(data_path + 'NPs/Gendered_NPs.csv')
all_names = names_df['Male'].to_list() + names_df['Female'].to_list()

In [55]:
nps_3m_df = pd.read_csv(data_path+'NPs/nps_3m.csv')
nps_3f_df = pd.read_csv(data_path+'NPs/nps_3f.csv')
nps_2m_f_df = pd.read_csv(data_path+'NPs/nps_2m_f.csv')
nps_2f_m_df = pd.read_csv(data_path+'NPs/nps_2f_m.csv')
nps_2m_df = pd.read_csv(data_path+'NPs/nps_2m.csv')
nps_2f_df = pd.read_csv(data_path+'NPs/nps_2f.csv')


In [56]:
np_3m = tuple(nps_3m_df.iloc[7,1:])
np_3f = tuple(nps_3f_df.iloc[7,1:])
np_2m_f = tuple(nps_2m_f_df.iloc[7,1:])
np_2f_m = tuple(nps_2f_m_df.iloc[7,1:])
np_2m = tuple(nps_2m_df.iloc[7,1:])
np_2f = tuple(nps_2f_df.iloc[7,1:])

# Generate data for human judgements test 

Type ECO -> Type1

Type ECS -> Type2

Type IC -> Type3

Type TOP -> Type4

In [197]:
col_names = ['Text','Type','Ambiguous','N3','NameA','NameB','Verb']
df_all = pd.DataFrame(columns = col_names)

In [198]:
sentences = []
NameAs = []
NameBs = []
Verbs = []

## Type ECO

In [199]:
def grammar(np_pair):
  np_pair_g = []
  index = 0
  for i in range(len(np_pair)):
    if np_pair[i] in all_names:
      np_pair_g.append(np_pair[i])

    else:
      if i == 0:
        np_pair_g.append("The " + np_pair[i])
      else:
        np_pair_g.append("the " + np_pair[i])
    
    index += 1

  return np_pair_g

### Unambiguous

####Unambiguous ECO-1 Active


In [200]:
sentences = []
NameAs = []
NameBs = []
Verbs = []
cutoff = int((len(see_verbs) + 1) / 2)

In [201]:
import random
import itertools

i = 0

for s in see_verbs[:cutoff]:
  i = i + 1
  Verbs.append(s)
  #f1 told f2 that she saw f3 (f3!=f1, f3!=f2, f1!=f2)
  if i % 4 == 0:
    f1, f2, f3 = grammar(np_3f)
    if "The " in f1 or "the " in f2:
     sentences.append(f1 + " told " + f2 + " that she " + s + " " + f3  +  ".\n") 
     NameAs.append(f1)
     NameBs.append(f2)


  if i % 4 == 1:
  #f1 told f2 that she saw m
    f1, f2, m = grammar(np_2f_m)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " told " + f2 + " that she " + s + " " + m  +  ".\n") 
      NameAs.append(f1)
      NameBs.append(f2)
    

  #m1 told m2 that he saw m3 (m3!=m1, m3!=m2, m1!=m2)
  if i % 4 == 2:
    m1,m2,m3 = grammar(np_3m)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " told " + m2 + " that he " + s + " " + m3  +  ".\n") 
      NameAs.append(m1)
      NameBs.append(m2)


  #m1 told m2 that she saw f
  if i % 4 == 3:
    m1,m2,f = grammar(np_2m_f)
    if "The " in m1 or "the " in m2:
     sentences.append(m1 + " told " + m2 + " that he " + s + " " + f  +  ".\n") 
     NameAs.append(m1)
     NameBs.append(m2)


In [202]:
df_temp = pd.DataFrame(zip(sentences,NameAs,NameBs,Verbs), columns=['Text','NameA','NameB','Verb'] )
df_temp['Type'] = 1
df_temp['Ambiguous'] = 0
df_temp['N3'] = 1
df_all = pd.concat([df_all, df_temp])

#### Unambiguous ECO-2 Active


In [203]:
sentences = []
NameAs = []
NameBs = []
Verbs = []

In [204]:
import random

i = 0
for s in see_verbs[cutoff:]:
  i = i + 1
  Verbs.append(s)
  if i % 2 == 0:
    #f1 told f2 that she saw <the client>
    f1,f2  = grammar(np_2f)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " told " + f2 + " that she " + s + " the client.\n")  
      NameAs.append(f1)
      NameBs.append(f2)

  else:
    m1,m2 = grammar(np_2m)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " told " + m2 + " that he " + s + " the client.\n") 
      NameAs.append(m1)
      NameBs.append(m2)


In [205]:
df_temp = pd.DataFrame(zip(sentences,NameAs,NameBs,Verbs), columns=['Text','NameA','NameB','Verb'] )
df_temp['Type'] = 1
df_temp['Ambiguous'] = 0
df_temp['N3'] = 0
df_all = pd.concat([df_all, df_temp])

In [206]:
df_temp

,Text,NameA,NameB,Verb,Type,Ambiguous,N3
0,The sister-in-law told Amanda that she intervi...,The sister-in-law,Amanda,interrogated,1,0,0
1,The sister-in-law told Amanda that she laughed...,The sister-in-law,Amanda,interviewed,1,0,0
2,The sister-in-law told Amanda that she liked t...,The sister-in-law,Amanda,invited,1,0,0
3,The sister-in-law told Amanda that she mentore...,The sister-in-law,Amanda,laughed at,1,0,0
4,The sister-in-law told Amanda that she misdire...,The sister-in-law,Amanda,led,1,0,0
5,The sister-in-law told Amanda that she mistrus...,The sister-in-law,Amanda,liked,1,0,0
6,The sister-in-law told Amanda that she mocked ...,The sister-in-law,Amanda,loved,1,0,0
7,The sister-in-law told Amanda that she negotia...,The sister-in-law,Amanda,mentored,1,0,0
8,The sister-in-law told Amanda that she played ...,The sister-in-law,Amanda,met,1,0,0
9,The sister-in-law told Amanda that she preferr...,The sister-in-law,Amanda,misdirected,1,0,0


In [207]:
sentences = []
NameAs = []
NameBs = []
Verbs = []

In [208]:
import random

i = 0
for s in see_verbs[cutoff:]:
  i = i + 1
  Verbs.append(s)
  if i % 2 == 0:
    #f1 told f2 that she saw <the client>
    f1,f2  = grammar(np_2f)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " was told by " + f2 + " that she " + s + " the client.\n")  
      NameAs.append(f1)
      NameBs.append(f2)

  else:
    m1,m2 = grammar(np_2m)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " was told by " + m2 + " that he " + s + " the client.\n") 
      NameAs.append(m1)
      NameBs.append(m2)
      


In [209]:
df_temp = pd.DataFrame(zip(sentences,NameAs,NameBs,Verbs), columns=['Text','NameA','NameB','Verb'] )
df_temp['Type'] = 1
df_temp['Ambiguous'] = 0
df_temp['N3'] = 0
df_all = pd.concat([df_all, df_temp])

###Ambiguous

#### Ambiguous ECO-1 Active


In [210]:
sentences = []
NameAs = []
NameBs = []
Verbs = []

In [211]:
import random

i = 0

for s in amuse_verbs[:cutoff]:
  i = i + 1
  Verbs.append(s)

  if i % 4 == 0:
    #f1 told f2 that she bored f3 (f3!=f1, f3!=f2, f1!=f2)
    f1, f2, f3 = grammar(np_3f)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " told " + f2 + " that she " + s + " " + f3  +  ".\n") 
      NameAs.append(f1)
      NameBs.append(f2)

  if i % 4 == 1:
    #f1 told f2 that she bored m
    f1,f2, m  = grammar(np_2f_m)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " told " + f2 + " that she " + s + " " + m  +  ".\n") 
      NameAs.append(f1)
      NameBs.append(f2)
  

  if i % 4 == 2:
    #m1 told m2 that he bored m3 (m3!=m1, m3!=m2, m1!=m2)
    m1,m2,m3 = grammar(np_3m)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " told " + m2 + " that he " + s + " " + m3  +  ".\n") 
      NameAs.append(m1)
      NameBs.append(m2)


  if i % 4 == 3:
    #m1 told m2 that she bored f
    m1,m2,f = grammar(np_2m_f)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " told " + m2 + " that he " + s + " " + f  +  ".\n") 
      NameAs.append(m1)
      NameBs.append(m2)


In [212]:
len(sentences)

27

In [213]:
df_temp = pd.DataFrame(zip(sentences,NameAs,NameBs,Verbs), columns=['Text','NameA','NameB','Verb'] )
df_temp['Type'] = 1
df_temp['Ambiguous'] = 1
df_temp['N3'] = 1
df_all = pd.concat([df_all, df_temp])

In [214]:
df_temp

,Text,NameA,NameB,Verb,Type,Ambiguous,N3
0,The son told Robert that he alarmed Brian.\n,The son,Robert,aggrieved,1,1,1
1,Kimberly told the aunt that she amazed the gra...,Kimberly,the aunt,alarmed,1,1,1
2,The son told Robert that he annoyed Brian.\n,The son,Robert,alienated,1,1,1
3,Kimberly told the aunt that she appalled the g...,Kimberly,the aunt,amazed,1,1,1
4,The son told Robert that he astonished Brian.\n,The son,Robert,amused,1,1,1
5,Kimberly told the aunt that she baffled the gr...,Kimberly,the aunt,annoyed,1,1,1
6,The son told Robert that he bewildered Brian.\n,The son,Robert,antagonized,1,1,1
7,Kimberly told the aunt that she bugged the gra...,Kimberly,the aunt,appalled,1,1,1
8,The son told Robert that he captivated Brian.\n,The son,Robert,appeased,1,1,1
9,Kimberly told the aunt that she concerned the ...,Kimberly,the aunt,astonished,1,1,1


#### Ambiguous ECO-2 Active


In [215]:
sentences = []
NameAs = []
NameBs = []
Verbs = []

In [216]:
import random

i = 0

for v in amuse_verbs[cutoff:]:
  i = i + 1
  Verbs.append(v)
  if i % 2 == 0:
    #f1 told f2 that she bored <the client>
    f1,f2 = grammar(np_2f)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " told " + f2 + " that she " + v + " the client.\n") 
      NameAs.append(f1)
      NameBs.append(f2)
        
  else:
    #m1 told m2 that he bored <the client>
    m1,m2 = grammar(np_2m)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " told " + m2 + " that he " + v + " the client.\n") 
      NameAs.append(m1)
      NameBs.append(m2)
        

In [217]:
df_temp = pd.DataFrame(zip(sentences,NameAs,NameBs,Verbs), columns=['Text','NameA','NameB','Verb'] )
df_temp['Type'] = 1
df_temp['Ambiguous'] = 1
df_temp['N3'] = 0
df_all = pd.concat([df_all, df_temp])

In [218]:
df_temp

,Text,NameA,NameB,Verb,Type,Ambiguous,N3
0,The sister-in-law told Amanda that she harmed ...,The sister-in-law,Amanda,gratified,1,1,0
1,The sister-in-law told Amanda that she hearten...,The sister-in-law,Amanda,harmed,1,1,0
2,The sister-in-law told Amanda that she humilia...,The sister-in-law,Amanda,haunted,1,1,0
3,The sister-in-law told Amanda that she impress...,The sister-in-law,Amanda,heartened,1,1,0
4,The sister-in-law told Amanda that she inspire...,The sister-in-law,Amanda,horrified,1,1,0
5,The sister-in-law told Amanda that she intimid...,The sister-in-law,Amanda,humiliated,1,1,0
6,The sister-in-law told Amanda that she irritat...,The sister-in-law,Amanda,hurt,1,1,0
7,The sister-in-law told Amanda that she misguid...,The sister-in-law,Amanda,impressed,1,1,0
8,The sister-in-law told Amanda that she moved t...,The sister-in-law,Amanda,infuriated,1,1,0
9,The sister-in-law told Amanda that she overwhe...,The sister-in-law,Amanda,inspired,1,1,0


## Type ECS

### Unambiguous

#### Unambiguous ECS-1 Active


In [219]:
sentences = []
NameAs = []
NameBs = []
Verbs = []

In [220]:
import random

i = 0
for s in meet_verbs[20:]:
  i = i + 1
  Verbs.append(s)
  if i % 4 == 0:
    #f1 told f2 that f3 met with her (f3!=f1, f3!=f2, f1!=f2)
    f1, f2, f3 = grammar(np_3f)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " told " + f2 + " that " + f3 + " " + s + " her.\n") 
      NameAs.append(f1)
      NameBs.append(f2)


  if i % 4 == 1:
    #f1 told f2 that m met with her
    f1,f2,m = grammar(np_2f_m)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " told " + f2 + " that " + m + " " + s + " her.\n") 
      NameAs.append(f1)
      NameBs.append(f2)


  if i % 4 == 2:
    #m1 told m2 that m3 met with him (m3!=m1, m3!=m2, m1!=m2)
    m1,m2,m3 = grammar(np_3m)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " told " + m2 + " that " + m3 + " " + s + " him.\n") 
      NameAs.append(m1)
      NameBs.append(m2)
      

  if i % 4 == 3:
    #m1 told m2 that f met with him
    m1,m2,f = grammar(np_2m_f)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " told " + m2 + " that " + f + " " + s + " him.\n") 
      NameAs.append(m1)
      NameBs.append(m2)
        

In [221]:
df_temp = pd.DataFrame(zip(sentences,NameAs,NameBs,Verbs), columns=['Text','NameA','NameB','Verb'] )
df_temp['Type'] = 2
df_temp['Ambiguous'] = 0
df_temp['N3'] = 1
df_all = pd.concat([df_all, df_temp])

In [222]:
df_temp

,Text,NameA,NameB,Verb,Type,Ambiguous,N3
0,The son told Robert that Brian excited him.\n,The son,Robert,encouraged,2,0,1
1,Kimberly told the aunt that the granddaughter ...,Kimberly,the aunt,excited,2,0,1
2,The son told Robert that Brian horrified him.\n,The son,Robert,exhausted,2,0,1
3,Kimberly told the aunt that the granddaughter ...,Kimberly,the aunt,fought with,2,0,1
4,The son told Robert that Brian interested him.\n,The son,Robert,frustrated,2,0,1
5,Kimberly told the aunt that the granddaughter ...,Kimberly,the aunt,horrified,2,0,1
6,The son told Robert that Brian offended him.\n,The son,Robert,hugged,2,0,1
7,Kimberly told the aunt that the granddaughter ...,Kimberly,the aunt,impressed,2,0,1
8,The son told Robert that Brian shouted at him.\n,The son,Robert,inspired,2,0,1
9,Kimberly told the aunt that the granddaughter ...,Kimberly,the aunt,interested,2,0,1


#### Unambiguous ECS-2 Active


In [223]:
sentences = []
NameAs = []
NameBs = []
Verbs = []

In [224]:
import random

i = 0

for s in meet_verbs[:20]:
  i = i + 1
  Verbs.append(s)
  if i % 2 == 0:
    #f1 told f2 that the client met with her (f3!=f1, f3!=f2, f1!=f2)
    f1, f2  = grammar(np_2f)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " told " + f2 + " that the client " + s + " her.\n") 
      NameAs.append(f1)
      NameBs.append(f2)
      

  else:
    #m1 told m2 that the client met with him
    m1, m2 = grammar(np_2m)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " told " + m2 + " that the client " + s + " him.\n") 
      NameAs.append(m1)
      NameBs.append(m2)
      

In [225]:
df_temp = pd.DataFrame(zip(sentences,NameAs,NameBs,Verbs), columns=['Text','NameA','NameB','Verb'] )
df_temp['Type'] = 2
df_temp['Ambiguous'] = 0
df_temp['N3'] = 0
df_all = pd.concat([df_all, df_temp])

In [226]:
df_temp

,Text,NameA,NameB,Verb,Type,Ambiguous,N3
0,The sister-in-law told Amanda that the client ...,The sister-in-law,Amanda,alarmed,2,0,0
1,The sister-in-law told Amanda that the client ...,The sister-in-law,Amanda,amazed,2,0,0
2,The sister-in-law told Amanda that the client ...,The sister-in-law,Amanda,amused,2,0,0
3,The sister-in-law told Amanda that the client ...,The sister-in-law,Amanda,annoyed,2,0,0
4,The sister-in-law told Amanda that the client ...,The sister-in-law,Amanda,bored,2,0,0
5,The sister-in-law told Amanda that the client ...,The sister-in-law,Amanda,bumped into,2,0,0
6,The sister-in-law told Amanda that the client ...,The sister-in-law,Amanda,calmed,2,0,0
7,The sister-in-law told Amanda that the client ...,The sister-in-law,Amanda,chatted with,2,0,0
8,The sister-in-law told Amanda that the client ...,The sister-in-law,Amanda,comforted,2,0,0
9,The sister-in-law told Amanda that the client ...,The sister-in-law,Amanda,complained to,2,0,0


### Ambiguous

#### Ambiguous ECS-1 Active


In [227]:
sentences = []
NameAs = []
NameBs = []
Verbs = []

In [228]:
import random

i = 0

for s in admire_verbs[20:]:
  i = i + 1
  Verbs.append(s)
  if i % 4 == 0:
    #f1 told f2 that f3 liked her (f3!=f1, f3!=f2, f1!=f2)
    f1,f2,f3  = grammar(np_3f)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " told " + f2 + " that " + f3 + " " + s + " her.\n") 
      NameAs.append(f1)
      NameBs.append(f2)
      
  if i % 4 == 1:
    #f1 told f2 that m liked her
    f1,f2,m  = grammar(np_2f_m)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " told " + f2 + " that " + m + " " + s + " her.\n")
      NameAs.append(f1)
      NameBs.append(f2)
      

  if i % 4 == 2:
    #m1 told m2 that m3 liked him (m3!=m1, m3!=m2, m1!=m2)
    m1,m2,m3 = grammar(np_3m)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " told " + m2 + " that " + m3 + " " + s + " him.\n") 
      NameAs.append(m1)
      NameBs.append(m2)

  if i % 4 == 3:
    #m1 told m2 that f liked him
    m1,m2,f = grammar(np_2m_f)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " told " + m2 + " that " + f + " " + s + " him.\n") 
      NameAs.append(m1)
      NameBs.append(m2)
      

In [229]:
df_temp = pd.DataFrame(zip(sentences,NameAs,NameBs,Verbs), columns=['Text','NameA','NameB','Verb'] )
df_temp['Type'] = 2
df_temp['Ambiguous'] = 1
df_temp['N3'] = 1
df_all = pd.concat([df_all, df_temp])

In [230]:
df_temp

,Text,NameA,NameB,Verb,Type,Ambiguous,N3
0,The son told Robert that Brian leaned on him.\n,The son,Robert,lamented to,2,1,1
1,Kimberly told the aunt that the granddaughter ...,Kimberly,the aunt,leaned on,2,1,1
2,The son told Robert that Brian misapprehended ...,The son,Robert,liked,2,1,1
3,Kimberly told the aunt that the granddaughter ...,Kimberly,the aunt,loathed,2,1,1
4,The son told Robert that Brian misunderstood h...,The son,Robert,loved,2,1,1
5,Kimberly told the aunt that the granddaughter ...,Kimberly,the aunt,misapprehended,2,1,1
6,The son told Robert that Brian relied on him.\n,The son,Robert,misinterpreted,2,1,1
7,Kimberly told the aunt that the granddaughter ...,Kimberly,the aunt,missed,2,1,1
8,The son told Robert that Brian supported him.\n,The son,Robert,mistrusted,2,1,1
9,Kimberly told the aunt that the granddaughter ...,Kimberly,the aunt,misunderstood,2,1,1


#### Ambiguous ECS-2 Active


In [231]:
sentences = []
NameAs = []
NameBs = []
Verbs = []

In [232]:
import random

i = 0

for s in admire_verbs[:20]:
  i = i + 1
  Verbs.append(s)
  if i % 2 == 0:
    #f1 told f2 that the client met with her (f3!=f1, f3!=f2, f1!=f2)
    f1, f2  = grammar(np_2f)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " told " + f2 + " that the client " + s + " her.\n") 
      NameAs.append(f1)
      NameBs.append(f2)
    

  else:
    #m1 told m2 that the client met with him
    m1, m2 = grammar(np_2m)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " told " + m2 + " that the client " + s + " him.\n") 
      NameAs.append(m1)
      NameBs.append(m2)
      

In [233]:
df_temp = pd.DataFrame(zip(sentences,NameAs,NameBs,Verbs), columns=['Text','NameA','NameB','Verb'] )
df_temp['Type'] = 2
df_temp['Ambiguous'] = 1
df_temp['N3'] = 0
df_all = pd.concat([df_all, df_temp])

## Type IC

###Ambiguous Active

In [234]:
sentences = []
NameAs = []
NameBs = []
Verbs = []

In [235]:
import random

i = 0

for c in call_verbs:
  i = i + 1
  for r in reasons_amb: 
    Verbs.append(r)
    if i % 2 == 0:
      m1, m2 = grammar(np_2m)
      if "The " in m1 or "the " in m2:
        sentences.append(m1 + " " + c + " " + m2 + " because he " + r + ".\n")
        NameAs.append(m1)
        NameBs.append(m2)

    else:
      f1, f2  = grammar(np_2f)
      if "The " in f1 or "the " in f2:
        sentences.append(f1 + " " + c + " " + f2 + " because she " + r + ".\n")
        NameAs.append(f1)
        NameBs.append(f2)

In [236]:
df_temp = pd.DataFrame(zip(sentences,NameAs,NameBs,Verbs), columns=['Text','NameA','NameB','Verb'] )
df_temp['Type'] = 3
df_temp['Ambiguous'] = 1
df_temp['N3'] = -1
df_all = pd.concat([df_all, df_temp])

In [237]:
df_temp

,Text,NameA,NameB,Verb,Type,Ambiguous,N3
0,The sister-in-law called Amanda because she is...,The sister-in-law,Amanda,is flying abroad soon,3,1,-1
1,The sister-in-law called Amanda because she is...,The sister-in-law,Amanda,is traveling abroad tomorrow,3,1,-1
2,The sister-in-law called Amanda because she is...,The sister-in-law,Amanda,is leaving the town soon,3,1,-1
3,The sister-in-law called Amanda because she is...,The sister-in-law,Amanda,is moving abroad soon,3,1,-1
4,The sister-in-law called Amanda because she is...,The sister-in-law,Amanda,is leaving town next week,3,1,-1
5,The sister-in-law called Amanda because she is...,The sister-in-law,Amanda,is attending a conference next week,3,1,-1
6,The sister-in-law called Amanda because she ha...,The sister-in-law,Amanda,has an important interview tomorrow,3,1,-1
7,The sister-in-law called Amanda because she ha...,The sister-in-law,Amanda,had been on leave for a long time,3,1,-1
8,The sister-in-law called Amanda because she wa...,The sister-in-law,Amanda,was upset,3,1,-1
9,The sister-in-law contacted Amanda because she...,The sister-in-law,Amanda,is flying abroad soon,3,1,-1


###Unambiguous Active

In [238]:
sentences = []
NameAs = []
NameBs = []
Verbs = []

In [239]:
import random

i = 0

for c in call_verbs:
  i = i + 1
  for r in reasons_unamb: 
    Verbs.append(r)
    if i % 2 == 0:
      m1, m2 = grammar(np_2m)
      if "The " in m1 or "the " in m2:
        sentences.append(m1 + " " + c + " " + m2 + " because he " + r + ".\n")
        NameAs.append(m1)
        NameBs.append(m2)
    else:
      f1, f2 = grammar(np_2f)
      if "The " in f1 or "the " in f2:
        sentences.append(f1 + " " + c + " " + f2 + " because she " + r + ".\n")
        NameAs.append(f1)
        NameBs.append(f2)
        

In [240]:
df_temp = pd.DataFrame(zip(sentences,NameAs,NameBs,Verbs), columns=['Text','NameA','NameB','Verb'] )
df_temp['Type'] = 3
df_temp['Ambiguous'] = 0
df_temp['N3'] = -1
df_all = pd.concat([df_all, df_temp])

In [241]:
df_temp

,Text,NameA,NameB,Verb,Type,Ambiguous,N3
0,The sister-in-law called Amanda because she wa...,The sister-in-law,Amanda,wanted to ask a question,3,0,-1
1,The sister-in-law called Amanda because she wa...,The sister-in-law,Amanda,wanted to share some updates,3,0,-1
2,The sister-in-law called Amanda because she ha...,The sister-in-law,Amanda,had a story to tell,3,0,-1
3,The sister-in-law called Amanda because she wa...,The sister-in-law,Amanda,wanted to apologize,3,0,-1
4,The sister-in-law called Amanda because she wa...,The sister-in-law,Amanda,was looking for suggestions,3,0,-1
5,The sister-in-law called Amanda because she wa...,The sister-in-law,Amanda,was seeking advice,3,0,-1
6,The sister-in-law called Amanda because she wa...,The sister-in-law,Amanda,wanted to catch up,3,0,-1
7,The sister-in-law called Amanda because she wa...,The sister-in-law,Amanda,wanted to get some clarification,3,0,-1
8,The sister-in-law called Amanda because she wa...,The sister-in-law,Amanda,was seeking feedback,3,0,-1
9,The sister-in-law contacted Amanda because she...,The sister-in-law,Amanda,wanted to ask a question,3,0,-1


## Type TOP

###Ambiguous Active

In [242]:
import random
sentences = []
NameAs = []
NameBs = []
Verbs = []

In [243]:
j = 0

for i,row in TOP_all.iterrows():
  j = j + 1
  verb_m = row['event_ambig']
  verb_f = male_to_female(verb_m)

  if j % 2 == 0:
    f1,f2  = grammar(np_2f)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " "+ row['passed'] + " " + f2 + " " + row['DO'] + " " + row['prep'] + " she " + verb_f + ".\n")
      Verbs.append(verb_f)
      NameAs.append(f1)
      NameBs.append(f2)
  else:
    m1,m2 = grammar(np_2m)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " "+ row['passed'] + " " + m2 + " " + row['DO'] + " " + row['prep'] + " he " + verb_m + ".\n")
      Verbs.append(verb_m)
      NameAs.append(m1)
      NameBs.append(m2)


In [244]:
df_temp = pd.DataFrame(zip(sentences,NameAs,NameBs,Verbs), columns=['Text','NameA','NameB','Verb'] )
df_temp['Type'] = 4
df_temp['Ambiguous'] = 1
df_temp['N3'] = -1
df_all = pd.concat([df_all, df_temp])

### Unambiguous Active

In [245]:
sentences = []
NameAs = []
NameBs = []
Verbs = []

In [246]:
j = 0

for i,row in TOP_all.iterrows():
  j = j + 1
  verb_m = row['event_unambig']
  verb_f = male_to_female(verb_m)

  if j % 2 == 0:
    f1,f2  = grammar(np_2f)
    if "The " in f1 or "the " in f2:
      sentences.append(f1 + " "+ row['passed'] + " " + f2 + " " + row['DO'] + " " + row['prep'] + " she " + verb_f + ".\n")
      Verbs.append(verb_f)
      NameAs.append(f1)
      NameBs.append(f2)

  else:
    m1,m2 = grammar(np_2m)
    if "The " in m1 or "the " in m2:
      sentences.append(m1 + " "+ row['passed'] + " " + m2 + " " + row['DO'] + " " + row['prep'] + " he " + verb_m + ".\n")
      Verbs.append(verb_m)
      NameAs.append(m1)
      NameBs.append(m2)


In [247]:
df_temp = pd.DataFrame(zip(sentences,NameAs,NameBs,Verbs), columns=['Text','NameA','NameB','Verb'] )
df_temp['Type'] = 4
df_temp['Ambiguous'] = 0
df_temp['N3'] = -1
df_all = pd.concat([df_all, df_temp])

In [248]:
df_temp

,Text,NameA,NameB,Verb,Type,Ambiguous,N3
0,The sister-in-law asked Amanda one more questi...,The sister-in-law,Amanda,ran out of time to answer more questions,4,0,-1
1,The sister-in-law asked Amanda about the detai...,The sister-in-law,Amanda,briefly introduced the plan to her,4,0,-1
2,The sister-in-law baked Amanda some egg tarts ...,The sister-in-law,Amanda,told her to do so,4,0,-1
3,The sister-in-law baked Amanda some banana bre...,The sister-in-law,Amanda,said that she wanted some,4,0,-1
4,The sister-in-law bought Amanda a drink after ...,The sister-in-law,Amanda,begged her to do so,4,0,-1
5,The sister-in-law bought Amanda a box of snack...,The sister-in-law,Amanda,finished the previous box,4,0,-1
6,The sister-in-law brought Amanda some flowers ...,The sister-in-law,Amanda,went to the flower shop,4,0,-1
7,The sister-in-law brought Amanda a gift after ...,The sister-in-law,Amanda,invited her to her birthday party,4,0,-1
8,The sister-in-law built Amanda a snowman after...,The sister-in-law,Amanda,said she wanted one,4,0,-1
9,The sister-in-law built Amanda a garden after ...,The sister-in-law,Amanda,expressed the desire of owning a garden,4,0,-1


## Export to csv

In [249]:
data_path

'/content/drive/My Drive/AmbiCoref/AmbiCoref/Data/'

In [250]:
df_all.to_csv(data_path + "Human_judgements/Turk-final-raw-final.csv")

# MTurk results analysis

## Get expected ground truth labels

In [281]:
import pandas as pd
turk_final_raw = pd.read_csv(data_path+'Human_judgements/Turk-final-raw-final.csv') 

In [282]:
turk_final_raw = turk_final_raw[['Text','Type','Ambiguous','N3']]

In [283]:
turk_final_raw_grouped = turk_final_raw.groupby(['Type','Ambiguous','N3'])    
turk_data_groups = [turk_final_raw_grouped.get_group(x) for x in turk_final_raw_grouped.groups]

In [284]:
type_name = {
    1 : "ECO",
    2 : "ECS",
    3 : "IC",
    4 : "TOP"
}

In [285]:
all_categories = []
df_turk_raw = pd.DataFrame(columns=['Text','Type','Ambiguous','N3','Category'])


for g in turk_data_groups:
  row = g.iloc[0,:]
  category = type_name[row['Type']]
  if row['N3'] == 0:
    category = category + "-2"
  elif row['N3'] == 1:
    category = category + "-1"
  if row['Ambiguous'] == 0:
    category = category + "_unamb"
  else:
    category = category + "_ambig"

  g['Category'] = category

  df_turk_raw = pd.concat([df_turk_raw,g]).sample(frac=1).reset_index(drop=True)

  if category not in all_categories:
    all_categories.append(category)
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [286]:
df_turk_raw = df_turk_raw[['Text','Category']]

In [287]:
def get_text_category(text):
  return df_turk_raw[df_turk_raw['Text']==text]['Category']

In [288]:
active_sentences = df_turk_raw['Text'].to_list()

## Analyze batch results

### Read batch results file from MTurk

In [289]:
relevant_columns = [
  'Answer.Q0',
  'Answer.Q1',
  'Answer.Q2',
  'Answer.Q3',
  'Answer.Q4',
  'Answer.Q5',
  'Answer.Q6',
  'Answer.Q7',
  'Answer.Q8',
  'Answer.Q9',
  'Answer.native',
  'Input.sentence0',
  'Input.sentence1',
  'Input.sentence2',
  'Input.sentence3',
  'Input.sentence4',
  'Input.sentence5',
  'Input.sentence6',
  'Input.sentence7',
  'Input.sentence8',
  'Input.sentence9'
]

In [290]:
import pandas as pd
results_df = pd.read_csv(data_path+"Human_judgements/final_batch_results.csv")
results_df = results_df[relevant_columns]

In [291]:
results_df

,Answer.Q0,Answer.Q1,Answer.Q2,Answer.Q3,Answer.Q4,Answer.Q5,Answer.Q6,Answer.Q7,Answer.Q8,Answer.Q9,...,Input.sentence0,Input.sentence1,Input.sentence2,Input.sentence3,Input.sentence4,Input.sentence5,Input.sentence6,Input.sentence7,Input.sentence8,Input.sentence9
0,prob-Q0-Name2,prob-Q1-Name2,prob-Q2-Name1,prob-Q3-Name2,prob-Q4-Name2,def-Q5-Name1,def-Q6-Name2,prob-Q7-Name2,prob-Q8-Name2,prob-Q9-Name1,...,Matthew bought Joshua a pizza after he asked f...,Matthew baked Joshua a cake before he asked fo...,Melissa told Jennifer that the father-in-law n...,Kimberly told the aunt that she baffled the gr...,Matthew was emailed by Joshua because he is le...,Matthew told Joshua that the client bored him.\n,Matthew showed Joshua the edited video before ...,Matthew was sent a message by Joshua because h...,The sister-in-law showed Amanda an example pic...,The sister-in-law was asked one more question ...
1,prob-Q0-Name1,prob-Q1-Name1,prob-Q2-Name1,prob-Q3-Name2,def-Q4-Name1,def-Q5-Name2,prob-Q6-Name2,prob-Q7-Name1,prob-Q8-Name2,$def-Q9-Name1,...,Matthew bought Joshua a pizza after he asked f...,Matthew baked Joshua a cake before he asked fo...,Melissa told Jennifer that the father-in-law n...,Kimberly told the aunt that she baffled the gr...,Matthew was emailed by Joshua because he is le...,Matthew told Joshua that the client bored him.\n,Matthew showed Joshua the edited video before ...,Matthew was sent a message by Joshua because h...,The sister-in-law showed Amanda an example pic...,The sister-in-law was asked one more question ...
2,prob-Q0-Name1,prob-Q1-Name2,def-Q2-Name1,prob-Q3-Name2,prob-Q4-Name1,def-Q5-Name1,def-Q6-Name1,def-Q7-Name2,def-Q8-Name1,prob-Q9-Name2,...,The sister-in-law called Amanda because she is...,Matthew was emailed by Joshua because he has a...,Matthew told Joshua that he rewarded the clien...,Kimberly told the aunt that the granddaughter ...,The sister-in-law contacted Amanda because she...,The sister-in-law told Amanda that the client ...,William told Joshua that the saleswoman visite...,Melissa was told by Jennifer that the father-i...,William told Joshua that he explained to the s...,Matthew told Joshua that he offended the clien...
3,Ambiguous,prob-Q1-Name1,def-Q2-Name1,prob-Q3-Name2,prob-Q4-Name1,def-Q5-Name1,def-Q6-Name1,prob-Q7-Name2,def-Q8-Name1,prob-Q9-Name2,...,The sister-in-law called Amanda because she is...,Matthew was emailed by Joshua because he has a...,Matthew told Joshua that he rewarded the clien...,Kimberly told the aunt that the granddaughter ...,The sister-in-law contacted Amanda because she...,The sister-in-law told Amanda that the client ...,William told Joshua that the saleswoman visite...,Melissa was told by Jennifer that the father-i...,William told Joshua that he explained to the s...,Matthew told Joshua that he offended the clien...
4,def-Q0-Name1,def-Q1-Name2,def-Q2-Name1,prob-Q3-Name2,def-Q4-Name1,def-Q5-Name1,def-Q6-Name1,def-Q7-Name2,def-Q8-Name1,prob-Q9-Name2,...,The sister-in-law called Amanda because she is...,Matthew was emailed by Joshua because he has a...,Matthew told Joshua that he rewarded the clien...,Kimberly told the aunt that the granddaughter ...,The sister-in-law contacted Amanda because she...,The sister-in-law told Amanda that the client ...,William told Joshua that the saleswoman visite...,Melissa was told by Jennifer that the father-i...,William told Joshua that he explained to the s...,Matthew told Joshua that he offended the clien...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,def-Q0-Name1,Ambiguous,Ambiguous,def-Q3-Name1,Ambiguous,def-Q5-Name1,def-Q6-Name2,prob-Q7-Name1,prob-Q8-Name1,prob-Q9-Name2,...,Noah told Liam that he begged Tristan.\n,Emma called Isabella because she must tell the...,Emma emailed Isabella because she is leaving t...,Emma emailed Isabella because she needed to ge...,Noah told Mason that he saddened Morgan.\n,Noah sent a message to Jacob because he wanted...,Emma showed Isabella the edi

### Count number of responses per each class for each category

#### Set up five classes of responses

In [292]:
responses = ["Name1, definitely", 
             "Name1, likely",
             "Ambiguous",
             "Name2, likely",
             "Name2, definitely"
             ]
# responses_print = ["Name1,\n definitely", 
#              "Name1,\n likely",
#              "Ambiguous",
#              "Name2,\n likely",
#              "Name2,\n definitely"
#              ]
responses_print = ["A", 
             "A?",
             "?",
             "B?",
             "B"
             ]

In [293]:
def get_response_class(answer):
  if "Name1" in answer:
    if "def" in answer:
      return "Name1, definitely"
    elif "prob" in answer:
      return "Name1, likely"
  elif "Name2" in answer:
    if "def" in answer:
      return "Name2, definitely"
    elif "prob" in answer:
      return "Name2, likely"
  else:
    return "Ambiguous"

#### Aggregate counts for each category

In [294]:
results_df.columns

Index(['Answer.Q0', 'Answer.Q1', 'Answer.Q2', 'Answer.Q3', 'Answer.Q4',
       'Answer.Q5', 'Answer.Q6', 'Answer.Q7', 'Answer.Q8', 'Answer.Q9',
       'Answer.native', 'Input.sentence0', 'Input.sentence1',
       'Input.sentence2', 'Input.sentence3', 'Input.sentence4',
       'Input.sentence5', 'Input.sentence6', 'Input.sentence7',
       'Input.sentence8', 'Input.sentence9'],
      dtype='object')

In [295]:
aggr_counts = dict()
for c in all_categories:
  aggr_counts[c] = dict(zip(responses, [0 for x in responses]))

In [296]:

for j,row in results_df.iterrows():
  
  for i in range(10):
    ans_col = 'Answer.Q'+str(i)
    response = get_response_class(row[ans_col])
    q_col = 'Input.sentence'+str(i)
    question = row[q_col]
    if question not in active_sentences:
      continue
    category = get_text_category(question).iloc[0]
  
    aggr_counts[category][response] += 1

## Generate analysis dataframe

In [297]:
analysis_df = pd.DataFrame(columns=responses+['Category'])

In [298]:
aggr_counts

{'ECO-2_unamb': {'Name1, definitely': 160,
  'Name1, likely': 86,
  'Ambiguous': 16,
  'Name2, likely': 32,
  'Name2, definitely': 27},
 'ECO-1_unamb': {'Name1, definitely': 183,
  'Name1, likely': 45,
  'Ambiguous': 8,
  'Name2, likely': 17,
  'Name2, definitely': 1},
 'ECO-2_ambig': {'Name1, definitely': 38,
  'Name1, likely': 36,
  'Ambiguous': 54,
  'Name2, likely': 106,
  'Name2, definitely': 22},
 'ECO-1_ambig': {'Name1, definitely': 31,
  'Name1, likely': 54,
  'Ambiguous': 60,
  'Name2, likely': 79,
  'Name2, definitely': 17},
 'ECS-2_unamb': {'Name1, definitely': 63,
  'Name1, likely': 21,
  'Ambiguous': 4,
  'Name2, likely': 2,
  'Name2, definitely': 0},
 'ECS-1_unamb': {'Name1, definitely': 81,
  'Name1, likely': 15,
  'Ambiguous': 5,
  'Name2, likely': 5,
  'Name2, definitely': 0},
 'ECS-2_ambig': {'Name1, definitely': 7,
  'Name1, likely': 4,
  'Ambiguous': 25,
  'Name2, likely': 47,
  'Name2, definitely': 7},
 'ECS-1_ambig': {'Name1, definitely': 15,
  'Name1, likely': 10

In [299]:
analysis_df = pd.DataFrame.from_dict(aggr_counts,orient='index')

In [300]:
analysis_df['Total'] = analysis_df[responses].sum(axis=1)

In [301]:
analysis_df.index.names = ['Category']

In [302]:
analysis_df

,"Name1, definitely","Name1, likely",Ambiguous,"Name2, likely","Name2, definitely",Total
Category,,,,,,
ECO-2_unamb,160,86,16,32,27,321
ECO-1_unamb,183,45,8,17,1,254
ECO-2_ambig,38,36,54,106,22,256
ECO-1_ambig,31,54,60,79,17,241
ECS-2_unamb,63,21,4,2,0,90
ECS-1_unamb,81,15,5,5,0,106
ECS-2_ambig,7,4,25,47,7,90
ECS-1_ambig,15,10,25,42,16,108
IC_unamb,259,111,34,22,5,431


In [303]:
analysis_df

,"Name1, definitely","Name1, likely",Ambiguous,"Name2, likely","Name2, definitely",Total
Category,,,,,,
ECO-2_unamb,160,86,16,32,27,321
ECO-1_unamb,183,45,8,17,1,254
ECO-2_ambig,38,36,54,106,22,256
ECO-1_ambig,31,54,60,79,17,241
ECS-2_unamb,63,21,4,2,0,90
ECS-1_unamb,81,15,5,5,0,106
ECS-2_ambig,7,4,25,47,7,90
ECS-1_ambig,15,10,25,42,16,108
IC_unamb,259,111,34,22,5,431


In [304]:
analysis_df.to_csv(data_path+'human_judgments_aggr1.csv')

In [305]:
data_path

'/content/drive/My Drive/AmbiCoref/AmbiCoref/Data/'